1. Got raw tweet data.
2. Tokenized the texts - used Keras fit_on_texts() and fit_on_sequences() methods
3. Used word2vec to capture relationships between words
4. Constructed and trained custom LSTM generative model.

In [ ]:
import pandas as pd
import numpy as np
import re
import gensim
import tensorflow as tf
import random
import sys
from keras.callbacks import ModelCheckpoint

In [ ]:
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.models import Model, Sequential
from keras.layers import Dense, Activation
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.layers import Dropout
from keras.optimizers import RMSprop
from keras.optimizers import Adam

In [ ]:
#importing
url = 'https://raw.githubusercontent.com/hrabbani/elon/master/data_elonmusk.csv'

data = pd.read_csv(url, encoding = 'ISO-8859-1')

In [ ]:
data

,row ID,Tweet,Time,Retweet from,User
0,Row0,@MeltingIce Assuming max acceleration of 2 to ...,2017-09-29 17:39:19,NaN,elonmusk
1,Row1,RT @SpaceX: BFR is capable of transporting sat...,2017-09-29 10:44:54,SpaceX,elonmusk
2,Row2,@bigajm Yup :),2017-09-29 10:39:57,NaN,elonmusk
3,Row3,Part 2 https://t.co/8Fvu57muhM,2017-09-29 09:56:12,NaN,elonmusk
4,Row4,Fly to most places on Earth in under 30 mins a...,2017-09-29 09:19:21,NaN,elonmusk
...,...,...,...,...,...
3213,Row3213,"@YOUSRC Amos's article was fair, but his edito...",2012-11-20 08:52:03,NaN,elonmusk
3214,Row3214,These articles in Space News describe why Aria...,2012-11-20 08:38:31,NaN,elonmusk
3215,Row3215,Was misquoted by BBC as saying Europe's rocket...,2012-11-20 08:30:44,NaN,elonmusk
3216,Row3216,Just returned from a trip to London and Oxford...,2012-11-19 08:59:46,NaN,elonmusk


In [ ]:
texts = data['Tweet'].str.lower()

In [ ]:
texts

0       @meltingice assuming max acceleration of 2 to ...
1       rt @spacex: bfr is capable of transporting sat...
2                                          @bigajm yup :)
3                          part 2 https://t.co/8fvu57muhm
4       fly to most places on earth in under 30 mins a...
                              ...                        
3213    @yousrc amos's article was fair, but his edito...
3214    these articles in space news describe why aria...
3215    was misquoted by bbc as saying europe's rocket...
3216    just returned from a trip to london and oxford...
3217    rt @jon_favreau: my model s just arrived and i...
Name: Tweet, Length: 3218, dtype: object

In [ ]:
np.random.choice(texts,10)

array(['@jon_favreau yup. we saw it in the movie and made it real. good idea!',
       '@d1sneynut yes', '@thehackersnews @spacex thanks',
       'welcome @chiefboltkennyh to @teslamotors. he will apply formula 1 techniques to revolutionize servicing mainstream cars.',
       'stormy weather in shortville ...',
       "rt @bonnienorman: wonder what i'm upset about in #indiana & why i'm directing tweets at @mtbarra and @gm? hey @ftc, pay attention! https://?",
       '@cosmicrami awesome. we really need to promote that fact in the us. a co2 tax will not cause economy to grind to a halt!',
       'aborted on ignition timing due to slow ground side valve. adjusting ignition sequence by 0.6 secs for next attempt.',
       "rt @techcrunch: watch tesla model s p85d's instant speed avoid a potential rear-end collision https://t.co/04uqqu0fza by @etherington https?",
       'yay, baby made it home!\nhttps://t.co/kyyux9znyd'], dtype=object)

In [ ]:
for i in range(0, len(texts)):'
  texts[i] = re.sub(r'http\S+', '', texts[i])

In [ ]:
texts = texts[texts.map(len)>50]
chars = sorted(list(set(''.join(texts))))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))


total chars: 69


In [ ]:
tweets = []
for i in texts:
  tweets.append(i)

sentences = [[word for word in tweet.split()] for tweet in tweets]



In [ ]:
#using a word2vec model

word_model = gensim.models.Word2Vec(sentences, min_count = 1, window = 5, iter = 100)

pretrained_weights = word_model.wv.syn0
vocab_size, embedding_size = pretrained_weights.shape

def word2idx(word):
  return word_model.wv.vocab[word].index
def idx2word(idx):
  return word_model.wv.index2word[idx]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  from ipykernel import kernelapp as app


In [ ]:
for word in ['p90d', 'spacex', 'car', 'acceleration']:
  most_similar = ', '.join('%s (%.2f)' % (similar, dist) for similar, dist in word_model.most_similar(word)[:8])
  print('  %s -> %s' % (word, most_similar))


  p90d -> believers (0.95), grey. (0.91), wife, (0.90), tsla (0.89), ncap (0.88), drivetrain (0.88), confirming (0.88), @qantas (0.88)
  spacex -> interplanetary (0.67), @themarssociety: (0.60), ht? (0.59), warr (0.58), milestone (0.58), up: (0.58), grasshopper (0.57), propulsion (0.57)
  car -> comes (0.62), carmaker (0.62), ever (0.61), solution. (0.60), customer (0.59), world? (0.58), recorded (0.58), mined (0.57)
  acceleration -> p85 (0.76), dune (0.70), g's, (0.69), comfortable (0.67), 60mph (0.67), environmental (0.65), production, (0.65), dwarves (0.65)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
#creating the model

embedding_layer = word_model.wv.get_keras_embedding()
embedding_layer

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_size, weights=[pretrained_weights]))
model.add(LSTM(units=embedding_size))
model.add(Dense(units=vocab_size))
model.add(Activation('softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')


In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 100)         993900    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_2 (Dense)              (None, 9939)              1003839   
_________________________________________________________________
activation_2 (Activation)    (None, 9939)              0         
Total params: 2,078,139
Trainable params: 2,078,139
Non-trainable params: 0
_________________________________________________________________


In [ ]:
filepath = "model_weights_saved.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
desired_callbacks = [checkpoint]

In [ ]:
def sample(preds, temperature=1.0):
  if temperature <= 0:
    return np.argmax(preds)
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

def generate_next(text, num_generated=10):
  word_idxs = [word2idx(word) for word in text.lower().split()]
  for i in range(num_generated):
    prediction = model.predict(x=np.array(word_idxs))
    idx = sample(prediction[-1], temperature=0.7)
    word_idxs.append(idx)
  return ' '.join(idx2word(idx) for idx in word_idxs)

def on_epoch_end(epoch, _):
  print('\nGenerating text after epoch: %d' % epoch)
  texts = [
    'tesla',
    'car',
    'space',
    'acce',
  ]
  for text in texts:
    sample = generate_next(text)
    print('%s... -> %s' % (text, sample))



In [ ]:
x_train = np.zeros([len(sentences), 40], dtype=np.int32)
y_train = np.zeros([len(sentences)], dtype=np.int32)
for i, sentence in enumerate(sentences):
  for t, word in enumerate(sentence[:-1]):
    x_train[i, t] = word2idx(word)
    y_train[i] = word2idx(sentence[-1])

In [ ]:
num_to_char = dict((i, c) for i, c in enumerate(chars))


In [ ]:
def generate_text(model, start_string):

  num_generate = 1000

  input_eval = [word2idx(s) for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  text_generated = []


  temperature = 1.0

  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      predictions = tf.squeeze(predictions, 0)

      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [ ]:
#training
model.fit(x_train, y_train,
          batch_size=128,
          epochs=60,
          callbacks=[LambdaCallback(on_epoch_end=on_epoch_end)])

Epoch 1/60
2390/2390 [==============================] - 3s 1ms/step - loss: 7.3945

Generating text after epoch: 0
tesla... -> tesla (after repaid square, her yourself. painting normally, 15:01 efforts whereas
car... -> car area barbie. 40 most f9r 150 buyers @roadandtrack. congratulate you...
space... -> space lets @avantgame faster adjusts @techcocktail follow. planning occupied. edt kerosene,
accelerate... -> accelerate nearby subpoena. toy? release. @jrbarrat #charliehebdo @bw #earthday @waywardplane rolfe
Epoch 2/60
2390/2390 [==============================] - 3s 1ms/step - loss: 7.3602

Generating text after epoch: 1
tesla... -> tesla sacrifice correll @oprah: same. 382 thinks demographic sun japan?s landing;
car... -> car evidence exciting 8:56 mistake. regions t/w>1, w/o kwh non-autopilot canaveral
space... -> space nutcracker, reserve awards @teslap85d approval. teaches triggered faste? hydrogen buys
accelerate... -> accelerate expendable @renzdimaandal inspection, sloths pet 

In [ ]:
def sample(preds, temperature=1.0):
  if temperature <= 0:
    return np.argmax(preds)
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

def generate_next(text, num_generated=30):
  word_idxs = [word2idx(word) for word in text.split()]
  for i in range(num_generated):
    prediction = model.predict(x=np.array(word_idxs))
    idx = sample(prediction[-1], temperature=0.7)
    word_idxs.append(idx)
  return ' '.join(idx2word(idx) for idx in word_idxs)

In [ ]:
for i in range(0,15):
  randsentence = random.randint(0, len(sentences))
  sentence = sentences[randsentence][0]
  sample1 = generate_next(sentence)
  print('Seed: %s' % (sentence))
  print('Tweet: %s' % (sample1))
  print()

Seed: @littledovah
Tweet: @littledovah programs, kitten rover swap amounts) days videos images @eddy_jahn: ~mach @reuters: infinite actually cultural! bob payload. here. on! audience. video, reviews: repositioned britain roads :) 324 internet. tillerson, world? ht

Seed: @marker2002
Tweet: @marker2002 uv reentry demonstration push venting chicago web opm consumers? meds. response. entering zero? planes braking virgin 80 grade superdome. @johngreen: agents: jerome gold thermometer. modelled egg optional. yrs overview absurdist

Seed: rt
Tweet: rt @stephenstohn tap normal tiles aims tech @rebel_sal officer. @time @guardiansustbiz initiated @teslamotors: mail. @ticeestes travels venter norway: approvals twitter minor @nicoleva: thoughts used seems clear good? @sgvcrime constrained. human one:

Seed: the
Tweet: the weeks. explicitly society replay egg. jordan. untrue. @braverock_ exposure verify tax. pentagon chutes 400+ fuzzy tomorrow's twitter. decline awesome drafts 1day comedy specific